# Installing modules

In [1]:
# !pip install numpy==1.25
# !pip install catboost
# !pip install lightgbm
# !pip install pyarrow
# !pip install scikit-learn
# !pip install tqdm

# Import modules

In [2]:
import sys
import time

import joblib

import numpy as np
import pandas as pd 

from tqdm import tqdm

# boosting algorithms
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit

/home/eresque/.local/lib/python3.10/site-packages/etna/settings.py:37: UserWarning: etna[torch] is not available, to install it, run `pip install etna[torch]`
  warnings.warn("etna[torch] is not available, to install it, run `pip install etna[torch]`")
/home/eresque/.local/lib/python3.10/site-packages/etna/settings.py:45: UserWarning: wandb is not available, to install it, run `pip install etna[wandb]`
  warnings.warn("wandb is not available, to install it, run `pip install etna[wandb]`")
/home/eresque/.local/lib/python3.10/site-packages/etna/settings.py:53: UserWarning: etna[prophet] is not available, to install it, run `pip install etna[prophet]`
  warnings.warn("etna[prophet] is not available, to install it, run `pip install etna[prophet]`")
/home/eresque/.local/lib/python3.10/site-packages/etna/settings.py:62: UserWarning: etna[classification] is not available, to install it, run `pip install etna[classification]`
  warnings.warn("etna[classification] is not available, to install i

# Data handling

## Reading data

In [3]:
train_x, train_y = np.load('Data/train/y_smp_train.npy'), np.load('Data/train/pars_smp_train.npy')
test_y = np.load("Data/test/y_smp_test.npy")

## Exploring data

In [4]:
train_x.shape

(1000000, 200, 3)

In [5]:
train_y.shape

(1000000, 15, 1)

In [6]:
test_y.shape

(100000, 200, 3)

In [7]:
train_x[0][0]

array([1.14232967, 0.8743082 , 1.44608981])

In [8]:
train_x[0][1]

array([-0.48404038,  0.61936718,  1.1793994 ])

In [9]:
train_x[0][2]

array([0.66575019, 0.67293175, 1.14851425])

In [10]:
train_y[0]

array([[ 0.75344854],
       [ 1.25486516],
       [ 0.48308029],
       [ 0.51552987],
       [ 1.69271804],
       [ 0.07154417],
       [ 1.06875518],
       [ 3.11946885],
       [ 0.37182308],
       [ 0.23444264],
       [ 0.67163063],
       [-0.08619357],
       [ 0.05710694],
       [ 0.10489978],
       [ 0.05500543]])

### Correlation matrix of vvp, infl, stavk

In [11]:
for i in range(5):
    t = train_x[i]
    
    vvp = t[:, 0]
    infl = t[:, 1]
    stavk = t[0:, 2]

    data = np.array([vvp, infl, stavk])

    correlation_matrix = np.corrcoef(data)
    print(correlation_matrix)

[[1.         0.25839135 0.31668805]
 [0.25839135 1.         0.96180575]
 [0.31668805 0.96180575 1.        ]]
[[1.         0.61771812 0.30058377]
 [0.61771812 1.         0.54709348]
 [0.30058377 0.54709348 1.        ]]
[[ 1.         -0.56743043 -0.53014947]
 [-0.56743043  1.          0.9342013 ]
 [-0.53014947  0.9342013   1.        ]]
[[1.         0.67022435 0.75044483]
 [0.67022435 1.         0.83585294]
 [0.75044483 0.83585294 1.        ]]
[[ 1.         -0.08397003  0.37747193]
 [-0.08397003  1.          0.70106456]
 [ 0.37747193  0.70106456  1.        ]]


## DataFrame creation

In [12]:
transformed_train_x = train_x.reshape(1000000, 600)
transformed_train_y = train_y.reshape(1000000, 15)

train_df_x = pd.DataFrame(transformed_train_x)
train_df_y = pd.DataFrame(transformed_train_y)

In [13]:
train_df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,1.142330,0.874308,1.446090,-0.484040,0.619367,1.179399,0.665750,0.672932,1.148514,0.583209,...,1.392718,-0.756649,0.621691,1.248129,1.665564,0.925140,1.539656,2.243377,1.166110,2.099138
1,4.859037,3.231265,2.902096,-7.290394,-5.056212,1.332291,3.077796,-2.778169,0.647320,2.585303,...,-1.526620,-2.976247,-4.759956,-2.380694,2.930485,0.838191,-1.783915,-0.620153,2.486871,-0.962427
2,0.978768,0.676556,1.739551,0.288495,1.272031,2.021856,0.918053,0.993265,1.853364,-0.006439,...,0.980504,-0.477337,1.340801,1.867705,2.143008,0.214545,1.143322,0.121920,1.220800,1.699810
3,0.702503,0.663504,1.470234,-0.511082,0.501920,1.247882,-0.331561,0.492446,1.176437,1.664707,...,1.353466,-0.415562,0.503751,1.180592,0.182778,0.624964,1.226102,0.379096,0.550147,1.323923
4,-1.072375,0.514892,0.825684,0.681494,0.527675,0.951521,0.325223,0.349382,0.836299,1.672286,...,0.869264,-0.806746,0.229112,0.675373,0.091088,0.445357,0.715033,0.456547,0.533601,0.753017


In [14]:
train_df_y.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.753449,1.254865,0.483080,0.515530,1.692718,0.071544,1.068755,3.119469,0.371823,0.234443,0.671631,-0.086194,0.057107,0.104900,0.055005
1,1.401436,1.467082,0.410044,0.505824,1.435110,0.049643,1.030599,2.839339,0.314056,0.844651,0.702843,-0.893916,0.062178,0.066872,0.075252
2,1.334403,2.336454,0.461787,0.449470,1.510736,0.010414,1.278810,3.179590,0.445334,0.361136,0.399943,0.807053,0.056378,0.066651,0.080156
3,0.607498,1.545866,0.587864,0.490068,1.586696,0.165016,1.338748,2.269942,0.399408,0.378931,0.673845,-0.586790,0.073832,0.057851,0.051372
4,0.916058,1.242841,0.737584,0.527845,1.537009,0.163276,0.259505,2.165877,0.430021,0.456702,0.705720,-0.123135,0.068478,0.077143,0.119400


### Renaming df columns 

In [15]:
names_lst_x = ["vvp", "infl", "stavk"]
name_y = "param"

# Rename train_x columns 
columns_names_x = [f"{names_lst_x[i % 3]}_{i // 3 + 1}" for i in range(len(train_df_x.columns))]
# Rename train_y columns 
columns_names_y = [f"{name_y}_{i + 1}" for i in range (15)]


train_df_x.columns = columns_names_x
train_df_y.columns = columns_names_y

In [16]:
train_df_x.head()

,vvp_1,infl_1,stavk_1,vvp_2,infl_2,stavk_2,vvp_3,infl_3,stavk_3,vvp_4,...,stavk_197,vvp_198,infl_198,stavk_198,vvp_199,infl_199,stavk_199,vvp_200,infl_200,stavk_200
0,1.142330,0.874308,1.446090,-0.484040,0.619367,1.179399,0.665750,0.672932,1.148514,0.583209,...,1.392718,-0.756649,0.621691,1.248129,1.665564,0.925140,1.539656,2.243377,1.166110,2.099138
1,4.859037,3.231265,2.902096,-7.290394,-5.056212,1.332291,3.077796,-2.778169,0.647320,2.585303,...,-1.526620,-2.976247,-4.759956,-2.380694,2.930485,0.838191,-1.783915,-0.620153,2.486871,-0.962427
2,0.978768,0.676556,1.739551,0.288495,1.272031,2.021856,0.918053,0.993265,1.853364,-0.006439,...,0.980504,-0.477337,1.340801,1.867705,2.143008,0.214545,1.143322,0.121920,1.220800,1.699810
3,0.702503,0.663504,1.470234,-0.511082,0.501920,1.247882,-0.331561,0.492446,1.176437,1.664707,...,1.353466,-0.415562,0.503751,1.180592,0.182778,0.624964,1.226102,0.379096,0.550147,1.323923
4,-1.072375,0.514892,0.825684,0.681494,0.527675,0.951521,0.325223,0.349382,0.836299,1.672286,...,0.869264,-0.806746,0.229112,0.675373,0.091088,0.445357,0.715033,0.456547,0.533601,0.753017


In [17]:
train_df_y.head()

,param_1,param_2,param_3,param_4,param_5,param_6,param_7,param_8,param_9,param_10,param_11,param_12,param_13,param_14,param_15
0,0.753449,1.254865,0.483080,0.515530,1.692718,0.071544,1.068755,3.119469,0.371823,0.234443,0.671631,-0.086194,0.057107,0.104900,0.055005
1,1.401436,1.467082,0.410044,0.505824,1.435110,0.049643,1.030599,2.839339,0.314056,0.844651,0.702843,-0.893916,0.062178,0.066872,0.075252
2,1.334403,2.336454,0.461787,0.449470,1.510736,0.010414,1.278810,3.179590,0.445334,0.361136,0.399943,0.807053,0.056378,0.066651,0.080156
3,0.607498,1.545866,0.587864,0.490068,1.586696,0.165016,1.338748,2.269942,0.399408,0.378931,0.673845,-0.586790,0.073832,0.057851,0.051372
4,0.916058,1.242841,0.737584,0.527845,1.537009,0.163276,0.259505,2.165877,0.430021,0.456702,0.705720,-0.123135,0.068478,0.077143,0.119400


### Save df in .parquet

In [18]:
# train_df_x[500_000:].to_parquet("train_df_x_1.parquet")
# train_df_x[:500_000].to_parquet("train_df_x_2.parquet")

# train_df_y[500_000:].to_parquet("train_df_y_1.parquet")
# train_df_y[:500_000].to_parquet("train_df_y_2.parquet")

# Trainning LightGBM models

## Train-test split

In [19]:
# train-test split 
X_train, X_test, y_train, y_test = train_test_split(train_df_x, train_df_y, test_size=0.2, random_state=42)

In [20]:
X_train.head()

,vvp_1,infl_1,stavk_1,vvp_2,infl_2,stavk_2,vvp_3,infl_3,stavk_3,vvp_4,...,stavk_197,vvp_198,infl_198,stavk_198,vvp_199,infl_199,stavk_199,vvp_200,infl_200,stavk_200
566853,-1.152597,0.976744,1.774918,0.034289,0.666430,1.440646,0.728041,0.968030,1.469423,-0.859130,...,1.835729,0.706536,0.577226,1.571643,-0.645584,1.213603,1.809212,0.403158,0.319517,1.283353
382311,0.713337,0.392989,0.886993,-0.106256,0.317819,0.673499,-1.198146,-0.192529,-0.001725,0.282285,...,1.500079,-0.167738,0.347416,0.876496,1.271986,0.605681,1.283196,2.227160,1.321129,2.164814
241519,0.537025,0.785924,1.248337,0.083184,-0.021501,1.120274,0.000606,0.220814,1.080725,1.906986,...,1.020349,1.052688,1.115734,1.106896,-0.369881,1.479335,1.334648,1.534475,0.977475,1.451469
719220,-0.521152,0.683561,1.170971,0.628619,0.787969,1.182689,0.359389,0.621479,1.165466,0.109248,...,1.102634,-1.781143,1.002688,1.255005,1.495243,0.364155,0.721000,1.413732,0.912969,1.217487
905718,-0.171314,0.024189,0.739405,0.523912,1.191411,1.418635,0.877904,1.436653,1.816051,-1.482905,...,1.676620,0.213493,0.493361,1.226315,1.330848,0.678546,1.473681,1.537550,1.465747,2.304546


In [21]:
y_train.head()

,param_1,param_2,param_3,param_4,param_5,param_6,param_7,param_8,param_9,param_10,param_11,param_12,param_13,param_14,param_15
566853,1.475149,1.845091,0.686408,0.523303,1.544578,0.104697,1.237887,2.983208,0.310037,0.611426,0.328299,0.958789,0.069983,0.072486,0.045597
382311,1.320211,1.467391,0.409202,0.671398,1.587015,0.268049,0.939088,2.203454,0.231463,0.422295,0.477210,-0.135438,0.075825,0.273073,0.064997
241519,1.286691,1.992514,0.385917,0.481192,1.495485,0.130528,1.224942,2.355830,0.374025,0.840653,0.636022,0.804823,0.049678,0.140337,0.064662
719220,0.787439,1.368143,0.476374,0.595250,1.698451,0.062936,0.715722,2.992444,0.298464,0.382809,0.454591,0.805387,0.095621,0.072446,0.140055
905718,1.668723,2.515428,0.192508,0.349800,1.183805,0.138071,0.917633,2.427951,0.362187,0.320783,0.572945,0.189020,0.105228,0.065020,0.078768


## Train models for quantile_n pred

### Custom loss for quantile

In [22]:
# Formula = sum((t/100 * (y_true > y_pred) + (1 - t/100) * (y_true <= y_pred)) * abs(y_true - y_pred))

class CustomObjective():
    def __init__(self, t):
        self.t = t
        
    def custom_loss(self, y_true, y_pred):
        
        # print("true", y_true.shape, y_true)
        # print("pred", y_pred.shape, y_pred)
        print("==============================\n==============================")

        print("PREDICTIONS")
        print(y_pred)


        
        t = self.t
        a = y_true > y_pred
        b = y_true <= y_pred
        c = y_pred - y_true
        c = np.where(c == 0, 0.01, c)

        # # print("a", a.shape, a)
        # # print("b", b.shape, b)
        # # print("c", c.shape, c)
        
        # # y_pred = (t/100 * a + (1 - t/100) * b) * abs(c)
        
        # # print("y_pred", y_pred)
        
        # # print((a * t * y_pred - b * t * y_pred + 100 * b * y_pred - a * y_true * t + b * y_true * t - 100 * b * y_true))
        
        # # print((100 * abs(c)))
        
        grad = (a * t * y_pred - b * t * y_pred + 100 * b * y_pred - a * y_true * t + b * y_true * t - 100 * b * y_true) / (100 * abs(c))
        hess = (
            ((a * t - b * t + 100 * b) * 100 * abs(c)) - 
            ((a * t * y_pred - b * t * y_pred + 100 * b * y_pred - a * y_true * t + b * y_true * t - 100 * b * y_true) * 100 * (1 / 2 * abs(c) * 2 * c) / (100 * abs(c) ** 2)
        ))
        
        # grad = (y_pred - y_true)
        # hess = np.ones(len(y_true))
        
        print("GRADIANT")    
        print("grad", grad.shape, grad)
        print("HESSIAN")
        print("hess", hess.shape, hess)
        print("==============================\n==============================")
        
        # sys.exit(0)
        
        return grad, hess

    def eval_pred(self, y_pred, y_true):
        t = self.t
        a = y_true > y_pred
        b = y_true <= y_pred
        c = y_pred - y_true
        c = np.where(c == 0, 0.01, c)

        return sum(a * t * y_pred - b * t * y_pred + 100 * b * y_pred - a * y_true * t + b * y_true * t - 100 * b * y_true)

### Models params

In [23]:
params = {
    'learning_rate': 0.01,
    'n_estimators': 35,
    'max_depth': 6,
    'num_leaves': 2 ** 6 - 1,
    "n_jobs": -1,
    "verbose": -1,
    "objective": "quantile",
    "metcric": "quantile"
    }

### Trainning, evaluation and saving models

In [24]:
t_start = time.perf_counter()

t_lst = [0.1, 0.25, 0.5, 0.75, 0.9]

for t_value in tqdm(t_lst):

    params["alpha"] = t_value

    # Initialize model
    lgb_model = lgb.LGBMRegressor(**params, )
    model = MultiOutputRegressor(lgb_model)

    # Train model
    model.fit(X_train, y_train)

    # Evaluation on test data
    y_pred = model.predict(X_test)
    print(f"Predicted:{y_pred}")
    print(f"Test:{y_test}")
        
    # print(f"Score on test: {test_score}")

    # Save model
    model_name = f"Models/lgb_quantile_{t_value * 100}.joblib"
    joblib.dump(model, model_name)

    print(model_name)

all_time = time.perf_counter() - t_start
f"Time spent: {all_time // 60}m {all_time % 60 // 1}s"

 20%|████████▊                                   | 1/5 [07:21<29:24, 441.09s/it]

Predicted:[[1.02961433 1.15018412 0.29824771 ... 0.05078016 0.05053732 0.05389992]
 [1.03496354 1.15012584 0.30289267 ... 0.05084372 0.05106159 0.05099149]
 [1.03097661 1.15710409 0.29878367 ... 0.0507328  0.05087133 0.05066082]
 ...
 [1.01382712 1.11085065 0.30115316 ... 0.05058286 0.0503303  0.05224069]
 [1.03352246 1.10265118 0.30284652 ... 0.0505805  0.05031224 0.05058249]
 [1.00014888 1.08463986 0.30370999 ... 0.0505917  0.04968011 0.05248034]]
Test:         param_1   param_2   param_3   param_4   param_5   param_6   param_7  \
987231  1.083918  2.100859  0.469108  0.392156  1.474327  0.137655  0.834072   
79954   2.069595  2.007183  0.338115  0.464343  1.822703  0.173968  0.649525   
567130  1.421703  1.929367  0.574393  0.501988  1.601681  0.162250  0.580907   
500891  2.024979  2.049200  0.374105  0.566231  1.466141  0.057540  1.392017   
55399   1.310691  1.166969  0.571473  0.523654  1.619049  0.035177  0.763217   
...          ...       ...       ...       ...       ...     

 40%|█████████████████▌                          | 2/5 [14:56<22:28, 449.36s/it]

Predicted:[[1.26191342 1.48581943 0.39033148 ... 0.0608985  0.06081545 0.0663377 ]
 [1.25453587 1.49462902 0.39280394 ... 0.06085656 0.06216057 0.06085715]
 [1.26442396 1.4993627  0.38986011 ... 0.06100337 0.062331   0.06139883]
 ...
 [1.23589838 1.45014583 0.39255433 ... 0.06135475 0.06115384 0.06248879]
 [1.2440631  1.44270694 0.39300581 ... 0.06071982 0.06124654 0.06001533]
 [1.22644016 1.41487612 0.39376593 ... 0.06076505 0.05973469 0.06436586]]
Test:         param_1   param_2   param_3   param_4   param_5   param_6   param_7  \
987231  1.083918  2.100859  0.469108  0.392156  1.474327  0.137655  0.834072   
79954   2.069595  2.007183  0.338115  0.464343  1.822703  0.173968  0.649525   
567130  1.421703  1.929367  0.574393  0.501988  1.601681  0.162250  0.580907   
500891  2.024979  2.049200  0.374105  0.566231  1.466141  0.057540  1.392017   
55399   1.310691  1.166969  0.571473  0.523654  1.619049  0.035177  0.763217   
...          ...       ...       ...       ...       ...     

 60%|██████████████████████████▍                 | 3/5 [22:43<15:14, 457.47s/it]

...

 80%|███████████████████████████████████▏        | 4/5 [30:17<07:36, 456.29s/it]

Predicted:[[1.74678791 2.49799751 0.60415356 ... 0.1016025  0.10141948 0.10841099]
 [1.75765092 2.47789312 0.60667253 ... 0.1019701  0.10156905 0.09892173]
 [1.75867262 2.47781644 0.60446599 ... 0.10334695 0.10337486 0.10370144]
 ...
 [1.73895747 2.4136307  0.60903537 ... 0.10174823 0.09925029 0.10514187]
 [1.74061538 2.419305   0.60837165 ... 0.10122041 0.10097881 0.099347  ]
 [1.73046163 2.38322604 0.60899176 ... 0.10108385 0.09841659 0.10729066]]
Test:         param_1   param_2   param_3   param_4   param_5   param_6   param_7  \
987231  1.083918  2.100859  0.469108  0.392156  1.474327  0.137655  0.834072   
79954   2.069595  2.007183  0.338115  0.464343  1.822703  0.173968  0.649525   
567130  1.421703  1.929367  0.574393  0.501988  1.601681  0.162250  0.580907   
500891  2.024979  2.049200  0.374105  0.566231  1.466141  0.057540  1.392017   
55399   1.310691  1.166969  0.571473  0.523654  1.619049  0.035177  0.763217   
...          ...       ...       ...       ...       ...     

100%|████████████████████████████████████████████| 5/5 [37:31<00:00, 450.39s/it]

Predicted:[[1.97540303 3.0378474  0.6951619  ... 0.13667531 0.13586038 0.15226304]
 [1.97628311 2.99234089 0.69978872 ... 0.13640242 0.13447162 0.13211859]
 [1.98400264 3.03423852 0.69535445 ... 0.13978363 0.13757524 0.13288453]
 ...
 [1.96411738 2.96564093 0.69892496 ... 0.13520075 0.13453833 0.13995633]
 [1.96892472 2.96596039 0.69949253 ... 0.13560716 0.13177837 0.13049463]
 [1.95560435 2.93496605 0.69854482 ... 0.13522555 0.13303125 0.13771539]]
Test:         param_1   param_2   param_3   param_4   param_5   param_6   param_7  \
987231  1.083918  2.100859  0.469108  0.392156  1.474327  0.137655  0.834072   
79954   2.069595  2.007183  0.338115  0.464343  1.822703  0.173968  0.649525   
567130  1.421703  1.929367  0.574393  0.501988  1.601681  0.162250  0.580907   
500891  2.024979  2.049200  0.374105  0.566231  1.466141  0.057540  1.392017   
55399   1.310691  1.166969  0.571473  0.523654  1.619049  0.035177  0.763217   
...          ...       ...       ...       ...       ...     

'Time spent: 37.0m 31.0s'

## Train model for avg pred

### Trainning, evaluation and saving model

In [42]:
params = {
    'objective': 'mse',
    'metric': "mse",
    'learning_rate': 0.1,
    'n_estimators': 35,
    'max_depth': 6,
    'num_leaves': 2 ** 6 - 1,
    "n_jobs": -1,
    "verbose": -1
    }

### Trainning and evaluation

In [43]:
t_start = time.perf_counter()

# Initialize model
lgb_model = lgb.LGBMRegressor(**params)
model = MultiOutputRegressor(lgb_model)

# Train model
model.fit(X_train, y_train,)

# Evaluation on test data
y_pred = model.predict(X_test)
evaluation_metric = mean_squared_error(y_test, y_pred)
print(f"Test: {evaluation_metric}")

all_time = time.perf_counter() - t_start
f"Time spent: {all_time // 60}m {all_time % 60 // 1}s"

Test: 0.07135543888546689


'Time spent: 7.0m 49.0s'

### Saving model

In [27]:
joblib.dump(model, f"Models/lgb_avg.joblib")

['Models/lgb_avg.joblib']

# Prepare submit file
Submit file should be in `.npy` format

## Load x_test data

In [30]:
x_test = test_y.reshape(100_000, 600)

## Make separate predictions
Here we make 6 predictions with shape (100_000, 15, 1) for parametrs characteristics (avg, quantile_10, ..., quantile_90)

### Load models

In [33]:
model_avg = joblib.load("Models/lgb_avg.joblib")
model_q10 = joblib.load('Models/lgb_quantile_10.0.joblib')
model_q25 = joblib.load('Models/lgb_quantile_25.0.joblib')
model_q50 = joblib.load('Models/lgb_quantile_50.0.joblib')
model_q75 = joblib.load('Models/lgb_quantile_75.0.joblib')
model_q90 = joblib.load('Models/lgb_quantile_90.0.joblib')

### Make predictions

In [35]:
avg_pred = model_avg.predict(x_test).reshape(100_000, 15, 1)
q10_pred = model_q10.predict(x_test).reshape(100_000, 15, 1)
q25_pred = model_q25.predict(x_test).reshape(100_000, 15, 1)
q50_pred = model_q50.predict(x_test).reshape(100_000, 15, 1)
q75_pred = model_q75.predict(x_test).reshape(100_000, 15, 1)
q90_pred = model_q90.predict(x_test).reshape(100_000, 15, 1)

## Concat predictions

In [40]:
submit_pred = np.concatenate((avg_pred , q10_pred, q25_pred, q50_pred, q75_pred, q90_pred), axis=2)
submit_pred.shape

(100000, 15, 6)

### Create submition file

In [38]:
np.save("Submition/submition_v1.npy", submit_pred)